In [ ]:
import numpy as np
import pandas as pd
import datetime
import pytz
#%load_ext autoreload

import pyarrow as pa
import pyarrow.parquet as pq

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
sleep = pd.read_parquet('Sleep_Mat.parquet', engine='pyarrow')
sleep['ID'] = sleep['patient_id'].apply(lambda x: x[:5])
sleep

In [ ]:
sleep['source'].unique()

In [ ]:
len(sleep['ID'].unique())

In [ ]:
def process_sleep_mat(patient_id, df):
    df = df[df['ID']==patient_id].reset_index(drop=True)
    df.loc[:,'start_date_tz'] = pd.to_datetime(df['start_date']).dt.tz_localize(tz='UTC')#
    df['duplicates']=0
    df = df.sort_values(by=['start_date_tz'])
    if df.duplicated(['start_date_tz']).sum()!=0:
        df['duplicates']=df.duplicated(['start_date_tz']).sum()
        print('Found some duplicate times, removing')
        df = df.drop_duplicates(subset=['start_date_tz'],keep='first')#
        
    return df[['ID','start_date_tz','state','snoring','duplicates','patient_id','start_date','heart_rate','respiratory_rate','source']]

In [ ]:
df = pd.DataFrame(columns=['ID','start_date_tz', 'state','snoring','duplicates','day','patient_id','value','start_date','heart_rate','respiratory_rate','duration_in_period'])
for i in sleep['ID'].unique():
        i_df = process_sleep_mat(i, sleep)
        df = pd.concat([df,i_df], ignore_index=True)
        print(i,len(df))

df

In [ ]:
df2 = pa.Table.from_pandas(df)
pq.write_table(df2, 'clean_sleep_data.parquet', compression='BROTLI')